In [ ]:
import os

from   matplotlib import pyplot as plt
import numpy as np
from   PIL import Image
import tensorflow as tf
from   tensorflow.keras import regularizers
from   tensorflow.keras.datasets import cifar10
from   tensorflow.keras.layers import (
    Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, 
    MaxPooling2D)
from   tensorflow.keras.models import load_model, Sequential
from   tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [1]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog',
          'horse', 'ship', 'truck']